<a href="https://colab.research.google.com/github/kaodamie/PAN-CLEF---Profilling-Hate-Spreaders-on-Twiter/blob/main/PAN21_Exp__Spanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q tensorflow-text

     |████████████████████████████████| 3.4MB 8.8MB/s 


In [ ]:
pip install -q tf-models-official

     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 37.6MB 1.3MB/s 
     |████████████████████████████████| 706kB 45.3MB/s 
     |████████████████████████████████| 174kB 45.1MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 1.2MB 37.8MB/s 
     |████████████████████████████████| 358kB 41.3MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 645kB 39.6MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os 
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as mpl
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import feature_column
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import shutil
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 
tf.get_logger().setLevel('ERROR')

name=r'/content/drive/MyDrive/Dataset/pan21-author-profiling-training-2021-03-14/es//'

xml_text=[]
list_filenames=[]
for file in os.listdir(name):
        filename = os.fsdecode(file)
        
        if filename.endswith('.xml'):
#             print(filename)   
            list_filenames.append(filename.split(sep='.')[0])
            try:
                tree = ET.parse(name+file) 
                root =tree.getroot() 
                xml_document=[]
                for neighbor in root.iter('document'):
                    xml_document.append(neighbor.text)
                     
                    #print(xml_document)
#                 print('\n\n')
                xml_text.append(" ".join(xml_document))
            except Exception as e:
                print(e, name+file)
#                 display(e, folder+file)
        
        elif filename.endswith('txt'):
            fileObject = open(name+file)
            data = fileObject.read().splitlines()
            dat=[]
            for i in data:
                dat.append(i.split(sep=':::'))
                

In [ ]:
df1=pd.DataFrame(zip(list_filenames,xml_text),columns=('ids','xml_text')).set_index('ids').join(pd.DataFrame(dat,columns=('ids','labels')).set_index(['ids']))
# v=pd.DataFrame(dat)
# v['xml'] =xml_text


# in future make a k-fold cross validation with sklearn
train, test = train_test_split(df1, test_size=0.2)
val, test = train_test_split(test, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train_ds=train['xml_text'].astype(str).to_numpy()
val_ds=val['xml_text'].astype(str).to_numpy()

whole_dataset=df1['xml_text'].astype(str).to_numpy()
whole_labels=df1['labels'].astype(int).to_numpy()

train_labels=train['labels'].astype(int).to_numpy()
val_labels=val['labels'].astype(int).to_numpy()

160 train examples
32 validation examples
8 test examples


In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_ds,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((val_ds,val_labels))

whole_ds=tf.data.Dataset.from_tensor_slices((whole_dataset,whole_labels))

train_ds = train_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
whole_ds=whole_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)

In [ ]:
#google sentence encoder classifier
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2",name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input) 
  encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2",name='encoder')

  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  # net = tf.keras.layers.Conv1D(1,1)(net)
  # net = tf.keras.layers.Flatten()(net)
  # net = tf.keras.layers.Activation('softmax')(net)
  # net = tf.keras.layers.Dropout(0.1)(net) 
  # net = tf.keras.layers.Conv1D(1,1)(net)
  # net = tf.keras.layers.Flatten()(net)
  # x2 = tf.keras.layers.Activation('softmax')(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
classifier_model.compile(optimizer='adam',
                         loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics=tf.metrics.BinaryAccuracy())

mcp_save = ModelCheckpoint('multi_sentence_encoder.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stop=EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [ ]:
#print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(train_ds,
                               validation_data=valid_ds,
                               epochs=10,callbacks=[mcp_save, early_stop], verbose=1)

#running with just adam was better than adamw /sentence encoder

Epoch 1/10
5/5 [==============================] - 62s 11s/step - loss: 0.6904 - binary_accuracy: 0.4963 - val_loss: 0.6819 - val_binary_accuracy: 0.4688
Epoch 2/10
5/5 [==============================] - 51s 11s/step - loss: 0.6516 - binary_accuracy: 0.5322 - val_loss: 0.6555 - val_binary_accuracy: 0.5312
Epoch 3/10
5/5 [==============================] - 51s 11s/step - loss: 0.6356 - binary_accuracy: 0.5208 - val_loss: 0.6391 - val_binary_accuracy: 0.5625
Epoch 4/10
5/5 [==============================] - 51s 11s/step - loss: 0.6161 - binary_accuracy: 0.5963 - val_loss: 0.6294 - val_binary_accuracy: 0.5312
Epoch 5/10
5/5 [==============================] - 51s 11s/step - loss: 0.6130 - binary_accuracy: 0.5776 - val_loss: 0.6216 - val_binary_accuracy: 0.5312
Epoch 6/10
5/5 [==============================] - 51s 11s/step - loss: 0.5845 - binary_accuracy: 0.5843 - val_loss: 0.6172 - val_binary_accuracy: 0.5312
Epoch 7/10
5/5 [==============================] - 51s 11s/step - loss: 0.5790 - bi

In [ ]:
# classifier_model.save('/content/drive/MyDrive/Dataset/PanResults/es')

In [ ]:
reloaded=tf.saved_model.load('/content/drive/MyDrive/Dataset/PanResults/es_savedmodel')

In [ ]:
reloaded

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f6a196e2610>

In [ ]:
train_loss, train_acc = classifier_model.evaluate(train_ds, verbose=1)
validation_loss, validation_acc =classifier_model.evaluate(valid_ds, verbose=1)

1/1 [==============================] - 9s 9s/step - loss: 0.5866 - binary_accuracy: 0.5625


In [ ]:
print('* Accuracy on training set: %0.2f%%' % (100 * train_acc))
print('* Accuracy on validation set: %0.2f%%' % (100 * validation_acc))

* Accuracy on training set: 47.50%
* Accuracy on validation set: 62.50%


In [ ]:

results=tf.sigmoid(classifier_model(tf.constant(test['xml_text'])))


results=tf.sigmoid(reloaded(tf.constant(test['xml_text'])))
#test_results=round()

In [ ]:
test['results']=results.numpy()

In [ ]:
#  a sample testset after shuffling
test 


,xml_text,labels,results
ids,,,
55225a39f4d2affd7494625a26f7dab2,y bueno fumando uno de los mios La verdad que ...,0,0.087965
9901087f781c97976d8af765bb14f80e,#USER# #USER# Dieta super calórica. #URL# #USE...,0,0.360244
6e3b4f88ca38d04437d315e3ec6e691f,RT #USER#: #USER# Asesinado por los mismos per...,1,0.626484
fa5be6e6c139255df5fabafa5812a062,RT #USER#: a dormir y recuerden que barovero l...,0,0.482056
20485577456acd57453192424391aae3,"A uno lo creen como marica, mor dale que no me...",0,0.244713
fa1a735295b654d2fceeead34dd71d96,"#USER# Desde arrimadas, ninguno. 🤔 ⁉️¿Por qué ...",1,0.744429
d0feaf7314acfc9272b75f936e12de3c,Esto es ya el colmo de los colmos. #URL# Cuand...,1,0.441626
0cc24f8fbc79eaf6dae74b1d42e365da,"RT #USER#: Felices 119 años, #USER#. Historia ...",1,0.616704


In [ ]:
# another sample test after shuffling
test

,xml_text,labels,results
ids,,,
b64385c12fad39e73e7018da60161db6,RT #USER#: ....vamos juntos a por los 75.........,1,0.600600
a802b5de3017f496053306f6d8be96b9,#USER# #USER# comparto cani ajajsjdhhshaajaj o...,0,0.400364
0af3ad19d2571df4cbca1948aba1019e,#USER# magnífico su resumen de esta semana 👏👏👏...,0,0.647418
b335bf6f5f8eda9979c489f28c3f48bc,RT #USER#: Gracias #USER# ! Gracias #USER# ! G...,0,0.383249
43a1b25e0fac45a86b5db1214e52fa28,#USER# Buena idea así revientan todos los Gori...,1,0.635178
59d66379e5c9b4a089c7d31e916a4c31,RT #USER#: makoke todos estamos contigo! Y sin...,0,0.577879
20485577456acd57453192424391aae3,"A uno lo creen como marica, mor dale que no me...",0,0.244713
b523abc590889dfaf69a1c8470ef314d,RT #USER#: - Sentite como en tu casa - Bueno m...,0,0.208792


In [ ]:
#working with the actual test data
name=r'/content/drive/MyDrive/Dataset/pan21-author-profiling-test-without-gold/es//'

xml_text=[]
list_filenames=[]
for file in os.listdir(name):
        filename = os.fsdecode(file)
        
        if filename.endswith('.xml'):
#             print(filename)   
            list_filenames.append(filename.split(sep='.')[0])
            try:
                tree = ET.parse(name+file) 
                root =tree.getroot() 
                xml_document=[]
                for neighbor in root.iter('document'):
                    xml_document.append(neighbor.text)
                     
                    #print(xml_document)
#                 print('\n\n')
                xml_text.append(" ".join(xml_document))
            except Exception as e:
                print(e, name+file)
#                 display(e, folder+file)

In [ ]:
df2=pd.DataFrame(zip(list_filenames,xml_text),columns=('ids','xml_text')).set_index('ids')

In [ ]:
df2

,xml_text
ids,
290a1bab7de4d014773e389b37375586,"RT #USER#: El ""pacto histórico"" a esta hora. #..."
1b95d9df96f463fb405d9718da1798fe,Se escribe viernes y se pronuncia al fin. #USE...
03263875c28c5e589409ab4ee1d18c2f,ee lo del concierto d ahora se puede ver por t...
03b9385278a1ec7bdf7e5775c138d7a0,Esto es: podemos organizarnos como proletariad...
0373317cba1543d5976c3a2cc2f92a79,"#USER# #USER# #USER# 11 seguidores, eres pejeb..."
...,...
ee15094987298282e6fe8f093cdf4d94,"#USER# #USER# Cómeme los huevos, subnormal Soi..."
fe3bd38287e375e1a3b61a1649837669,#USER# Era parte de la estrategia de forestal ...
d4161c2729e99219f215ff98f4ad9793,RT #USER#: ¿Se deberían convocar elecciones cu...


In [ ]:
# if reusing saved model, 
test_results=tf.sigmoid(reloaded(tf.constant(df2['xml_text'])))

# if not reusing saved model then comment out top
test_results=tf.sigmoid(classifier_model(tf.constant(df2['xml_text'])))

In [ ]:
for i in range(len(df2['xml_text'])):
  print(f'{df2.index[i]}: {round(test_results[i][0].numpy())}')
  root = ET.Element('author', id=df2.index[i],lang='es',type=f'{round(test_results[i][0].numpy())}')
  s = ET.tostring(root)
  print(str(s)) 
  tree = ET.ElementTree(root)
  # change directory
  tree.write("/content/drive/MyDrive/Dataset/PanResults/Test_Results/es/%s.xml" %df2.index[i])

290a1bab7de4d014773e389b37375586: 1
b'<author id="290a1bab7de4d014773e389b37375586" lang="es" type="1" />'
1b95d9df96f463fb405d9718da1798fe: 1
b'<author id="1b95d9df96f463fb405d9718da1798fe" lang="es" type="1" />'
03263875c28c5e589409ab4ee1d18c2f: 0
b'<author id="03263875c28c5e589409ab4ee1d18c2f" lang="es" type="0" />'
03b9385278a1ec7bdf7e5775c138d7a0: 1
b'<author id="03b9385278a1ec7bdf7e5775c138d7a0" lang="es" type="1" />'
0373317cba1543d5976c3a2cc2f92a79: 1
b'<author id="0373317cba1543d5976c3a2cc2f92a79" lang="es" type="1" />'
19f3683f4f99dfa0de0e4e937e7e3f28: 0
b'<author id="19f3683f4f99dfa0de0e4e937e7e3f28" lang="es" type="0" />'
0794ff63d194999ef7dfbb27fb67830c: 0
b'<author id="0794ff63d194999ef7dfbb27fb67830c" lang="es" type="0" />'
15f18af6d4b4008f87be4bdc68898c23: 1
b'<author id="15f18af6d4b4008f87be4bdc68898c23" lang="es" type="1" />'
1d6b3543165c1e22b7db4ecfb1fff4a4: 0
b'<author id="1d6b3543165c1e22b7db4ecfb1fff4a4" lang="es" type="0" />'
17abfb488b6ca1cfc52310709d811de1: 1
b